In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import IPython

def display_prediction(text, predictions):
    class ObjectToDisplay(object):
        def _repr_html_(self):
            cmap = matplotlib.cm.get_cmap('Reds')
            html_chars = []
            for c, p in zip(text, predictions):
                if c == '\n':
                    html_chars.append('<br>')
                else:
                    r, g, b, a = cmap(p)
                    r, g, b = int(256*r), int(256*g), int(256*b)
                    html_chars.append('<span style="background-color:rgb(%s, %s, %s); color:black;">%s</span>' % (r, g, b, c))
            tot_html = "".join(html_chars)
            return tot_html
    return ObjectToDisplay()

text = 'polska krowa \n spalinowa'
targets = np.random.random(len(text))
display_prediction(text, targets)

In [ ]:
import numpy as np

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, TimeDistributed
from keras.datasets import imdb
# from keras.models import Input

In [ ]:
import tensorflow as tf

In [ ]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [ ]:
def gen_text(n):
    chars = ['a']
    labels = [0]
    transitions = {
        'a': {
            'a': 0.10,
            'b': 0.37,
            'c': 0.53
        },
        'b': {
            'a': 0.2,
            'b': 0.6,
            'c': 0.2
        },
        'c': {
            'a': 0.1,
            'b': 0.2,
            'c': 0.7
        }
    }
    letters = ['a','b','c']

    for i in range(n-1):
        curr = chars[-1]
        tr = transitions[curr]
        nex = np.random.choice(letters, p=np.array([tr[c] for c in letters]))
        chars.append(nex)
        label = 1 if (chars[-2:] == ['a','c'] or chars[-2:] == ['b','b']) else 0
        labels.append(label)
    return chars, labels

In [ ]:
chars = '\n !"#$%&\'()*+,-./0123456789:;<=>?@[\\]^_`abcdefghijklmnopqrstuvwxyz{|}~U'
chars = 'abc'
charset = set(chars)
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
def encode_text(text):
    zeros = [[0] * len(chars) for _ in range(len(text))]
    for i, c in enumerate(text):
        zeros[i][char_indices[c]] = 1
    return zeros

In [ ]:
X = []
y = []
n = 100
maxlen=5
for i in range(n):
    text, labels = gen_text(maxlen)
    X.append(encode_text(text))
    y.append(labels)
    
X = np.array(X)
y = np.array(y).reshape((n, maxlen, 1))

In [ ]:
n_train = 80
X_train = X[:n_train, :, :]
y_train = y[:n_train, :, :]
X_test = X[n_train:, :, :]
y_test = y[n_train:, :, :]

In [ ]:
rnn_size = 5

batch_size = 1
n_chars = len(chars)
epochs = 16

model = Sequential()
model.add(LSTM(rnn_size, return_sequences=True, batch_input_shape=(batch_size, maxlen, n_chars)))
# model.add(LSTM(rnn_size, return_sequences=True, batch_input_shape=(batch_size, maxlen, n_chars)))
# model.add(Dropout(0.2))
# model.add(Dense(units=2, activation='sigmoid'))
model.add(TimeDistributed(Dense(units=1, activation='sigmoid')))
model.compile(optimizer='adam', loss='mse', metrics=['accuracy', 'binary_crossentropy'])

model.summary()
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
# model.evaluate(X_test, y_test, batch_size=batch_size)

In [ ]:
preds = model.predict(X_train, batch_size=batch_size).ravel()

In [ ]:
len(preds)

In [ ]:
len(X_train)

In [ ]:
import theano